<a href="https://colab.research.google.com/github/Sathwika2202/NLP/blob/main/2403A52250_BACH_09_ASSIGNMENT_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

# Load dataset
df = pd.read_excel("/content/LDA-Data.xlsx")

# Display first few rows
df.head()


,News
0,Virat scored century in match
1,BJP won in elections
2,Bumra took 5 wicket in a match
3,Congress form state government


In [5]:
# Extract text column (adjust column name if needed)
corpus = df['News'].astype(str).tolist()

print("Number of documents:", len(corpus))

Number of documents: 4


In [6]:
print(corpus[0])


Virat scored century in match


In [7]:
import re

def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text


In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = clean_text(text)
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word)
              for word in tokens if word not in stop_words and len(word) > 2]
    return tokens


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [9]:
# Apply preprocessing
processed_docs = [preprocess_text(doc) for doc in corpus]

print(processed_docs[0][:20])


['virat', 'scored', 'century', 'match']


In [10]:
processed_text = [' '.join(tokens) for tokens in processed_docs]

print(processed_text[0])


virat scored century match


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(
    max_df=0.95,      # ignore very frequent words
    min_df=1          # ignore rare words - adjusted for small dataset
)

bow = vectorizer.fit_transform(processed_text)

In [13]:
from sklearn.decomposition import LatentDirichletAllocation

num_topics = 5   # you can change this

lda = LatentDirichletAllocation(
    n_components=num_topics,
    random_state=42
)

lda.fit(bow)


LatentDirichletAllocation(n_components=5, random_state=42)

In [14]:
def display_topics(model, feature_names, num_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"\nTopic {topic_idx + 1}:")
        print(" ".join([
            feature_names[i]
            for i in topic.argsort()[:-num_top_words - 1:-1]
        ]))


In [15]:
display_topics(
    lda,
    vectorizer.get_feature_names_out(),
    num_top_words=10
)



Topic 1:
election bjp bumra took wicket virat scored century match state

Topic 2:
wicket took bumra scored virat century election bjp match state

Topic 3:
wicket took bumra scored virat century election bjp match state

Topic 4:
state form congress government scored virat century took wicket bumra

Topic 5:
match scored virat century wicket bumra took bjp election state


In [17]:
topic_distribution = lda.transform(bow)

df['Dominant_Topic'] = topic_distribution.argmax(axis=1)

df[['News', 'Dominant_Topic']].head()

,News,Dominant_Topic
0,Virat scored century in match,4
1,BJP won in elections,0
2,Bumra took 5 wicket in a match,4
3,Congress form state government,3


In [18]:
import pandas as pd
import numpy as np
import re

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [19]:
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
df = pd.read_csv("/content/arxiv_data.csv", engine='python', on_bad_lines='skip')

In [24]:
# Display first 5 rows
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


In [29]:
corpus = df['summaries'].astype(str).tolist()

print("Number of documents:", len(corpus))
print("\nSample document:\n", corpus[0])

Number of documents: 37664

Sample document:
 Stereo matching is one of the widely used techniques for inferring depth from
stereo images owing to its robustness and speed. It has become one of the major
topics of research since it finds its applications in autonomous driving,
robotic navigation, 3D reconstruction, and many other fields. Finding pixel
correspondences in non-textured, occluded and reflective areas is the major
challenge in stereo matching. Recent developments have shown that semantic cues
from image segmentation can be used to improve the results of stereo matching.
Many deep neural network architectures have been proposed to leverage the
advantages of semantic segmentation in stereo matching. This paper aims to give
a comparison among the state of art networks both in terms of accuracy and in
terms of speed which are of higher importance in real-time applications.


In [30]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text


In [31]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = clean_text(text)
    tokens = text.split()
    tokens = [
        lemmatizer.lemmatize(word)
        for word in tokens
        if word not in stop_words and len(word) > 2
    ]
    return tokens


In [32]:
processed_docs = [preprocess_text(doc) for doc in corpus]

print("Processed tokens (sample):")
print(processed_docs[0][:15])


Processed tokens (sample):
['stereo', 'matching', 'one', 'widely', 'used', 'technique', 'inferring', 'depth', 'stereo', 'image', 'owing', 'robustness', 'speed', 'become', 'one']


In [33]:
processed_text = [' '.join(tokens) for tokens in processed_docs]

print("Rejoined text (sample):")
print(processed_text[0])


Rejoined text (sample):
stereo matching one widely used technique inferring depth stereo image owing robustness speed become one major topic research since find application autonomous driving robotic navigation reconstruction many field finding pixel correspondence nontextured occluded reflective area major challenge stereo matching recent development shown semantic cue image segmentation used improve result stereo matching many deep neural network architecture proposed leverage advantage semantic segmentation stereo matching paper aim give comparison among state art network term accuracy term speed higher importance realtime application


In [34]:
vectorizer = CountVectorizer(max_df=0.95, min_df=5)
bow = vectorizer.fit_transform(processed_text)

print("BoW Matrix Shape:", bow.shape)


BoW Matrix Shape: (37664, 15537)


In [35]:
bow_df = pd.DataFrame(
    bow.toarray(),
    columns=vectorizer.get_feature_names_out()
)

print(bow_df.head())


   aae  aaf  aam  aan  abandon  abbr  abbreviate  abbreviated  abc  abdomen  \
0    0    0    0    0        0     0           0            0    0        0   
1    0    0    0    0        0     0           0            0    0        0   
2    0    0    0    0        0     0           0            0    0        0   
3    0    0    0    0        0     0           0            0    0        0   
4    0    0    0    0        0     0           0            0    0        0   

   ...  zinc  zone  zoo  zoom  zoomed  zooming  zsd  zsi  zsl  zurich  
0  ...     0     0    0     0       0        0    0    0    0       0  
1  ...     0     0    0     0       0        0    0    0    0       0  
2  ...     0     0    0     0       0        0    0    0    0       0  
3  ...     0     0    0     0       0        0    0    0    0       0  
4  ...     0     0    0     0       0        0    0    0    0       0  

[5 rows x 15537 columns]


In [36]:
lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(bow)

print("LDA model trained successfully")


LDA model trained successfully


In [38]:
def display_topics(model, feature_names, top_n):
    for idx, topic in enumerate(model.components_):
        print(f"\nTopic {idx + 1}:")
        print(" ".join(feature_names[i]
              for i in topic.argsort()[:-top_n - 1:-1]))

display_topics(lda, vectorizer.get_feature_names_out(), 10)



Topic 1:
learning model data representation method network adversarial generative distribution training

Topic 2:
learning image data segmentation model method deep task dataset network

Topic 3:
image method domain model network feature generative adversarial propose task

Topic 4:
object detection feature method network image graph segmentation proposed propose

Topic 5:
model network video neural transformer task architecture method performance accuracy


In [40]:
topic_dist = lda.transform(bow)

df['Dominant_Topic'] = topic_dist.argmax(axis=1)

print(df[['summaries', 'Dominant_Topic']].head())

                                           summaries  Dominant_Topic
0  Stereo matching is one of the widely used tech...               3
1  The recent advancements in artificial intellig...               1
2  In this paper, we proposed a novel mutual cons...               1
3  Consistency training has proven to be an advan...               2
4  To ensure safety in automated driving, the cor...               1
